In [1]:
# Setup: install libs (safe to re-run)
!pip -q install -U plotly kaleido matplotlib seaborn pandas

import plotly.express as px
import plotly.io as pio
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def save_plotly_png_failsafe(fig, filename):
    """Try to save Plotly figure to PNG.
    1) fig.write_image(..., engine='kaleido')
    2) fig.to_image(format='png')
    3) fallback to HTML
    Returns the saved filename (png or html).
    """
    # Attempt 1
    try:
        fig.write_image(filename, engine="kaleido")
        print(f"✅ PNG exported (kaleido): {filename}")
        return filename
    except Exception as e1:
        print("\n⚠️ kaleido failed:\n", e1, "\n")
    # Attempt 2
    try:
        png_bytes = fig.to_image(format="png")
        with open(filename, "wb") as f:
            f.write(png_bytes)
        print(f"✅ PNG exported (to_image): {filename}")
        return filename
    except Exception as e2:
        print("\n⚠️ to_image failed:\n", e2, "\n")
    # Attempt 3: HTML
    html_name = filename.replace(".png", ".html")
    pio.write_html(fig, html_name, include_plotlyjs="cdn", full_html=True)
    print(f"💾 Saved HTML fallback: {html_name}")
    return html_name

def ensure_static_fallback_from_dataframe(kind, df, outfile_png):
    """When Plotly PNG cannot be produced (e.g., environment lacks Chrome),
    create a static PNG via matplotlib/seaborn that approximates the Plotly figure.
    kind: 'scatter', 'heatmap', 'scatter_matrix'
    """
    try:
        if kind == 'scatter':
            plt.figure(figsize=(7,5))
            sns.scatterplot(data=df, x="GDP per capita", y="Score", hue="Country or region", s=40, linewidth=0)
            plt.title("Happiness Score vs GDP per Capita (static)")
            plt.tight_layout()
            plt.savefig(outfile_png, dpi=150, bbox_inches='tight')
            plt.close()
        elif kind == 'heatmap':
            plt.figure(figsize=(6,5))
            corr = df.select_dtypes('number').corr()
            sns.heatmap(corr, annot=True, fmt=".2f", cmap="viridis")
            plt.title("Correlation heatmap (static)")
            plt.tight_layout()
            plt.savefig(outfile_png, dpi=150, bbox_inches='tight')
            plt.close()
        elif kind == 'scatter_matrix':
            # Use seaborn pairplot
            g = sns.pairplot(df[["GDP per capita","Score","Overall rank"]], corner=False, diag_kind="hist")
            g.fig.suptitle("Scatter Matrix (static)", y=1.02)
            g.savefig(outfile_png, dpi=150, bbox_inches='tight')
            plt.close('all')
        print(f"🖼️ Static PNG fallback created: {outfile_png}")
        return outfile_png
    except Exception as ee:
        print("❌ Failed to create static fallback:", ee)
        return None



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


## Build demo dataframe (`happy_df`) from `gapminder` (2007)

In [2]:
gm = px.data.gapminder().query("year == 2007").copy()
happy_df = pd.DataFrame({
    "Country or region": gm["country"],
    "GDP per capita": gm["gdpPercap"],
    "Score": gm["lifeExp"],  # proxy for score
    "Overall rank": gm["pop"].rank(ascending=False).astype(int)
})
happy_df.head()


,Country or region,GDP per capita,Score,Overall rank
11,Afghanistan,974.580338,43.828,36
23,Albania,5937.029526,76.423,116
35,Algeria,6223.367465,72.301,35
47,Angola,4797.231267,42.731,64
59,Argentina,12779.379640,75.320,29


## Scatter: Happiness Score vs GDP per Capita

In [3]:
fig = px.scatter(
    happy_df, x="GDP per capita", y="Score",
    size="Overall rank", color="Country or region",
    hover_name="Country or region",
    title="Happiness Score vs GDP per Capita"
)
# annotate Taiwan/Japan/first row
target = "Taiwan" if (happy_df["Country or region"] == "Taiwan").any() else ("Japan" if (happy_df["Country or region"] == "Japan").any() else happy_df.iloc[0]["Country or region"])
row = happy_df.loc[happy_df["Country or region"] == target].iloc[0]
fig.add_annotation(x=row["GDP per capita"], y=row["Score"], text=target, showarrow=True, arrowhead=2, ax=20, ay=-40)

fig.show()
out = save_plotly_png_failsafe(fig, "fig1.png")
# if HTML fallback, also make a static PNG approximation
if out.endswith('.html'):
    ensure_static_fallback_from_dataframe('scatter', happy_df, 'fig1_static.png')



⚠️ kaleido failed:
 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome

 


⚠️ to_image failed:
 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome

 

💾 Saved HTML fallback: fig1.html
🖼️ Static PNG fallback created: fig1_static.png


![fig1](fig1.png)

*If PNG failed above, a static fallback may be saved as `fig1_static.png`.*

## Correlation heatmap

In [4]:
corr = happy_df.select_dtypes('number').corr()
fig = px.imshow(corr, text_auto=True, aspect='auto', title='Correlation heatmap')
fig.update_layout(margin=dict(l=40, r=20, t=40, b=40))
fig.show()
out = save_plotly_png_failsafe(fig, 'fig2.png')
if out.endswith('.html'):
    ensure_static_fallback_from_dataframe('heatmap', happy_df, 'fig2_static.png')



⚠️ kaleido failed:
 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome

 


⚠️ to_image failed:
 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome

 

💾 Saved HTML fallback: fig2.html
🖼️ Static PNG fallback created: fig2_static.png


![fig2](fig2.png)

*If PNG failed above, a static fallback may be saved as `fig2_static.png`.*

## Scatter matrix

In [5]:
fig = px.scatter_matrix(
    happy_df,
    dimensions=["GDP per capita", "Score", "Overall rank"],
    color="Country or region",
    title="Scatter Matrix"
)
fig.update_traces(diagonal_visible=False)
fig.show()
out = save_plotly_png_failsafe(fig, 'fig3.png')
if out.endswith('.html'):
    ensure_static_fallback_from_dataframe('scatter_matrix', happy_df, 'fig3_static.png')



⚠️ kaleido failed:
 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome

 


⚠️ to_image failed:
 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome

 

💾 Saved HTML fallback: fig3.html
🖼️ Static PNG fallback created: fig3_static.png


![fig3](fig3.png)

*If PNG failed above, a static fallback may be saved as `fig3_static.png`.*